# Model class

In [1]:
import os

os.chdir("../..")

In [2]:
!pwd

/Users/wiktorlazarski/Desktop/my_project/head-segmentation


In [3]:
import matplotlib.pyplot as plt

import head_segmentation.image_processing as ip
import head_segmentation.model as mdl

plt.rcParams["figure.figsize"] = (12, 10)
plt.rcParams["image.cmap"] = "gray"

from scripts.training import augmentations as aug
from scripts.training import data_loading as dl

%load_ext autoreload
%autoreload 2

In [4]:
DATASET_PATH = os.path.join(os.getcwd(), "data", "dataset", "train")

aug_pipeline = aug.AugmentationPipeline()

dataset = dl.CelebAHeadSegmentationDataset(
    dataset_path=DATASET_PATH, 
    augmentation_pipeline=aug_pipeline,
    preprocess_pipeline=ip.PreprocessingPipeline(nn_input_image_size=256),
)

In [5]:
model = mdl.HeadSegmentationModel(encoder_name="mobilenet_v2", encoder_depth=3, pretrained=False, image_input_size=256)

In [6]:
img, segmap = dataset[123]

In [7]:
type(segmap)

torch.Tensor

In [8]:
img.shape

torch.Size([3, 256, 256])

In [9]:
segmap.shape

torch.Size([256, 256])

In [10]:
segmap.dtype

torch.float32

In [11]:
segmap.max()

tensor(1.)

In [12]:
segmap.min()

tensor(0.)

In [13]:
mock_batch = img.unsqueeze(dim=0)

mdl_output = model.forward(mock_batch)

In [14]:
mdl_output.shape

torch.Size([1, 2, 256, 256])

In [15]:
# Model output to loss function
mdl_output

tensor([[[[ 0.1061,  0.3572, -0.1695,  ...,  0.7886,  0.2738, -0.6616],
          [ 0.3440,  0.3193, -0.1945,  ...,  0.4638,  0.1313, -0.8503],
          [ 0.3715,  0.8726,  0.4938,  ...,  0.9531,  1.0478, -0.5673],
          ...,
          [ 0.1936,  0.1107, -0.7079,  ...,  1.1708,  0.2294,  0.2454],
          [ 0.0359, -0.6889, -1.3152,  ...,  0.8416,  0.3473,  0.2718],
          [ 0.3506,  0.5247,  0.4904,  ..., -0.0204, -0.5503,  0.0905]],

         [[ 0.3761, -0.0261, -0.0872,  ...,  0.1861, -0.2054,  0.0608],
          [ 0.7309, -0.9523, -0.7721,  ...,  0.2791, -0.1216,  0.1023],
          [ 0.1903, -0.9336, -0.6644,  ..., -0.5766, -1.0841, -0.1290],
          ...,
          [-0.9497, -1.9179, -2.1169,  ..., -0.0806,  0.3768, -0.1364],
          [-0.7205, -1.6336, -1.7209,  ..., -0.1652,  0.6517,  0.6184],
          [-0.3703, -0.3052, -0.2408,  ...,  0.1565,  0.5470,  0.1804]]]],
       grad_fn=<ThnnConv2DBackward>)

In [16]:
def postprocessing(model_output):
    model_output = model_output.squeeze()
    segmap = model_output.argmax(dim=0)
    
    return segmap

In [17]:
mdl_output = postprocessing(mdl_output)

In [18]:
mdl_output.shape

torch.Size([256, 256])

In [19]:
mdl_output

tensor([[1, 0, 1,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])

In [20]:
model

HeadSegmentationModel(
  (encoder): MobileNetV2Encoder(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 96, kernel_si